<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/tutorials/1-Introduction/China_A_share_market_tushare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [1]:
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


Install other libraries

In [2]:
# !pip install stockstats
# !pip install tushare
# #install talib
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') 
# !./configure --prefix=/usr
# !make
# !make install
# #!sudo make install # Sometimes it need root 
# os.chdir('../')
# !pip install TA-Lib

In [3]:
# %cd /
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
# %cd /FinRL-Meta/

##Import Modules

In [4]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd 
from IPython import display

display.set_matplotlib_formats("svg")

from meta import config 
from meta.data_processor import DataProcessor 
from main import check_and_make_directories 
from meta.data_processors.tushare import Tushare, ReturnPlotter 
from meta.data_processors.tushare_private import ReturnPlotterPrivate
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv 
from agents.stablebaselines3_models import DRLAgent 
import os 
from typing import List 
from argparse import ArgumentParser 
from meta import config 
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


##Create Folders

In [5]:
import os

''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

##Download data, cleaning and feature engineering

In [6]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2020-01-03'

TIME_INTERVAL = "1d" 
kwargs = {} 
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

tushare successfully connected


###Download and Clean

In [7]:
p.download_data(ticker_list=ticker_list)

p.clean_data()

100%|██████████| 15/15 [00:08<00:00,  1.78it/s]


Shape of DataFrame:  (17960, 8)
Shape of DataFrame:  (18315, 8)


###Add technical indicator

In [8]:
p.add_technical_indicator(config.INDICATORS) 
p.clean_data()

#print(f"p.dataframe: {p.dataframe}")

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


##Split training dataset

In [9]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [10]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']


In [48]:
print(f"train.head(): {train.head()}")
# print(train.to_string())
# print(f"train.index: {train.index}")
# print(f"train.index.unique(): {train.index.unique()}")

            tic        date  index     open     high      low    close       volume  day       macd      boll_ub     boll_lb      rsi_30        cci_30       dx_30  close_30_sma  close_60_sma
0     600000.SH  2015-01-08     45    15.87    15.88    15.20    15.25   3306271.72  3.0  -0.032571    16.617911   15.012089    6.058641 -1.255930e+02   23.014040     15.815000     15.815000
0     600009.SH  2015-01-08     46    20.18    20.18    19.73    20.00    198117.45  3.0  -0.016008    20.663897   19.736103   12.828915 -9.084249e+01  100.000000     20.200000     20.200000
0     600016.SH  2015-01-08     47    10.61    10.66    10.09    10.20   4851684.17  3.0  -0.018247    10.957604    9.997396   11.862558 -9.988701e+01  100.000000     10.477500     10.477500
0     600028.SH  2015-01-08     48     7.09     7.41     6.83     6.85   8190902.35  3.0  -0.008227     7.342000    6.743000   27.409248  3.657817e+01   64.934862      7.042500      7.042500
0     600030.SH  2015-01-08     49    36.40  

In [49]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [50]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


##Train

In [51]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [52]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


###DDPG

In [53]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000)

NameError: name 'DRLAgent' is not defined

###A2C

In [ ]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_7
---------------------------------------
| time/                 |             |
|    fps                | 146         |
|    iterations         | 100         |
|    time_elapsed       | 3           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -21.3       |
|    explained_variance | -0.0032     |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -32.4       |
|    reward             | -0.24292411 |
|    std                | 1           |
|    value_loss         | 4.11        |
---------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 144      |
|    iterations         | 200      |
|    time_elapsed       | 6        |
|    total_timesteps    | 1000     |
| train/                | 

##Trade

In [ ]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)
# import pickle
# with open('./df_account_value.pickle', 'wb') as handle:
#     pickle.dump(df_account_value, handle, protocol=pickle.HIGHEST_PROTOCOL)


Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1153142.32
total_reward: 153142.32
total_cost: 68.68
total_trades: 618
Sharpe: 2.310
hit end!


In [ ]:
print(df_account_value)

           date  account_value
0    2019-08-01   1.000000e+06
1    2019-08-02   9.972021e+05
2    2019-08-05   9.826644e+05
3    2019-08-06   9.825871e+05
4    2019-08-07   9.830335e+05
..          ...            ...
99   2019-12-26   1.120953e+06
100  2019-12-27   1.125883e+06
101  2019-12-30   1.139063e+06
102  2019-12-31   1.151556e+06
103  2020-01-02   1.153142e+06

[104 rows x 2 columns]


In [ ]:
df_actions.to_csv("action.csv", index=False) 
print(f"df_actions: {df_actions}")

df_actions:             600000.SH  600009.SH  600016.SH  600028.SH  600030.SH  600031.SH  \
date                                                                           
2019-08-01          0       1000       1000          0          0          0   
2019-08-02          0       1000       1000          0          0          0   
2019-08-05          0       1000       1000          0          0          0   
2019-08-06          0        700       1000          0          0          0   
2019-08-07          0          0        100          0          0          0   
...               ...        ...        ...        ...        ...        ...   
2019-12-25          0          0          0          0          0          0   
2019-12-26          0          0          0          0          0          0   
2019-12-27          0          0          0          0          0          0   
2019-12-30          0          0          0          0          0          0   
2019-12-31          0       

##Backtest

###matplotlib inline

In [ ]:
import importlib

import meta.data_processors.tushare_private
importlib.reload(meta.data_processors.tushare_private)
from meta.data_processors.tushare_private import ReturnPlotterPrivate, TICKET_TYPE_INDEX, TICKET_TYPE_TICKET

from matplotlib import pyplot as plt

plotter = ReturnPlotterPrivate(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot(figure_filepath="./China_A_share.png")
# plotter.plot()

plt.close('all')

In [ ]:
# ticket: SSE 50：000016.SH
baseline_ticket = "000016.SH"
plotter.plot(baseline_ticket, figure_filepath="./China_A_share_vs_{0}.png".format(baseline_ticket), ticket_type=TICKET_TYPE_INDEX)
plt.close('all')

TICKET_TYPE_INDEX


###CSI 300

In [ ]:
baseline_df = plotter.get_baseline("399300.SZ", ticket_type=TICKET_TYPE_INDEX)
print("baseline_df ", baseline_df)

TICKET_TYPE_INDEX
baseline_df         ts_code trade_date      close       open       high        low  \
0    399300.SZ   20200103  4144.9649  4161.2185  4164.2989  4131.8640   
1    399300.SZ   20200102  4152.2408  4121.3487  4172.6555  4121.3487   
2    399300.SZ   20191231  4096.5821  4077.7519  4098.1444  4069.0086   
3    399300.SZ   20191230  4081.6334  4015.5195  4083.6901  4001.4951   
4    399300.SZ   20191227  4022.0278  4029.2454  4066.7964  4019.7223   
..         ...        ...        ...        ...        ...        ...   
100  399300.SZ   20190807  3621.4310  3654.6323  3659.0778  3621.4310   
101  399300.SZ   20190806  3636.3289  3609.1118  3649.8953  3575.8626   
102  399300.SZ   20190805  3675.6884  3724.4097  3739.5031  3675.6884   
103  399300.SZ   20190802  3747.4379  3729.1309  3754.5326  3720.0584   
104  399300.SZ   20190801  3803.4694  3819.3242  3831.2641  3791.5460   

     pre_close   change  pct_chg          vol       amount   dt  \
0    4152.2408  -7.2759  

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

==============DRL Strategy Stats===========
perf_stats_all: Annual return          0.412365
Cumulative returns     0.153142
Annual volatility      0.156216
Sharpe ratio           2.310304
Calmar ratio           7.048378
Stability              0.639828
Max drawdown          -0.058505
Omega ratio            1.480388
Sortino ratio          3.789312
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.079636
Daily value at risk   -0.018249
Alpha                       NaN
Beta                        NaN
dtype: float64


In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")

==============Baseline Strategy Stats===========
perf_stats_all: Annual return         -0.186455
Cumulative returns    -0.082388
Annual volatility      0.132137
Sharpe ratio          -1.510562
Calmar ratio          -1.458535
Stability              0.531397
Max drawdown          -0.127837
Omega ratio            0.786930
Sortino ratio         -2.015469
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.055412
Daily value at risk   -0.017440
Alpha                  0.000000
Beta                   1.000000
dtype: float64
